In [ ]:
from datetime import datetime, timedelta
from pathlib import Path
import os

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

DATA_DIR = Path('../../Data/Wind Turbine')

In [ ]:
import sys
sys.path.append('../')
from source.common import get_sensor_data_info


In [ ]:
os.environ['INPUT_DATA_DIR'] = '/media/dmitriy/D/Projects/PredictiveMaintenance/Data'

In [ ]:
pd.set_option('display.max_columns', 100)

## First look at data

### SCADA data

In [ ]:
scada_df = pd.read_csv(DATA_DIR / 'scada_data.csv')
scada_df['DateTime'] = pd.to_datetime(scada_df['DateTime'], format='%m/%d/%Y %H:%M')
scada_df['HasError'] = (scada_df['Error'] != 0).astype(int)
original_columns = scada_df.columns

In [ ]:
scada_df['DateTimeR'] = scada_df['DateTime'].dt.round(freq='10min')
# scada_df_gr = scada_df.groupby('DateTimeR', as_index=False).mean()

date_range = pd.Series(pd.date_range(start=scada_df['DateTimeR'].min(), end=scada_df['DateTimeR'].max(), freq='10min'), name='DateTimeR')
scada_df_gr = scada_df.merge(date_range, how='outer', on='DateTimeR').sort_values('DateTimeR')
scada_df_gr['HasMissing'] = (scada_df_gr['Time'].isna()).astype(int)

date_range_10s = pd.Series(pd.date_range(start=scada_df_gr['DateTimeR'].min(), end=scada_df_gr['DateTimeR'].max(), freq='10s'), name='DateTime10s')
scada_df_gr = scada_df_gr.merge(date_range_10s, how='outer', left_on='DateTimeR', right_on='DateTime10s').sort_values('DateTime10s').ffill().reset_index(drop=True)
scada_df_gr.loc[scada_df_gr['HasMissing'] == 1, original_columns] = np.nan
scada_df_gr.head()


In [ ]:
len(scada_df.columns)

In [ ]:
[tpl for tpl in zip(scada_df['DateTime'], scada_df['Error'])]

### Faults

In [ ]:
fault_df = pd.read_csv(DATA_DIR / 'fault_data.csv')
fault_df['DateTime'] = pd.to_datetime(fault_df['DateTime'], format='%Y-%m-%d %H:%M:%S')
fault_df['TimeDiff'] = fault_df['DateTime'] - fault_df['DateTime'].shift(1)
fault_df.head(30)


In [ ]:
fault_df['DateTimeR'] = fault_df['DateTime'].dt.round(freq='10s')

grouped_records = []
for dt, group_df in fault_df.groupby('DateTimeR', as_index=False):
    fault_record = (dt, ','.join(group_df['Fault'].unique()))
    grouped_records.append(fault_record)

grouped_fault_df = pd.DataFrame.from_records(grouped_records, columns=['DateTime', 'Faults'])

for fault_type in fault_df['Fault'].unique():
    grouped_fault_df[f'Fault_{fault_type}'] = (grouped_fault_df['Faults'].str.contains(fault_type)).astype(int)

# grouped_fault_df.head(50)

# date_range = pd.Series(pd.date_range(start=scada_df_gr['DateTimeR'].min(), end=scada_df_gr['DateTimeR'].max(), freq='10min'), name='DateTime')
# grouped_fault_df = grouped_fault_df.merge(date_range, how='outer', on='DateTime').sort_values('DateTime')
# grouped_fault_df['HasMissing'] = (grouped_fault_df['Faults'].isna()).astype(int)

grouped_fault_df['TimeSincePrevFault'] = grouped_fault_df['DateTime'] - grouped_fault_df['DateTime'].shift(1)
grouped_fault_df['FaultGroupID'] = (grouped_fault_df['TimeSincePrevFault'] > timedelta(hours=6)).astype(int)
grouped_fault_df['FaultGroupID'] = grouped_fault_df['FaultGroupID'].cumsum()
grouped_fault_df.head(60)


In [ ]:
fault_df['Fault'].value_counts()

### Status data

In [ ]:
status_df = pd.read_csv(DATA_DIR / 'status_data.csv')
status_df['Time'] = pd.to_datetime(status_df['Time'], format='%d/%m/%Y %H:%M:%S')
status_df = status_df.rename(columns={'Time': 'DateTime'})
status_df['DateTime10s'] = status_df['DateTime'].dt.round(freq='10s')
status_df['StatusRecordID'] = status_df.index
status_df['StateDurationTD'] = status_df['DateTime'].shift(-1) - status_df['DateTime']
status_df['StateDurationMinutes'] = status_df['StateDurationTD'].dt.total_seconds() / 60
status_df.head()

In [ ]:
status_df['Status Text'].unique()

In [ ]:
status_df.head(980).tail(40)

In [ ]:

date_range = pd.Series(pd.date_range(start=status_df['DateTime10s'].min(), end=status_df['DateTime10s'].max(), freq='10s'), name='DateTime10s')
status_df_ts = status_df[['DateTime10s', 'Main Status', 'Sub Status', 'StatusRecordID']]\
    .drop_duplicates(subset=['DateTime10s'], keep='last')\
    .merge(date_range, how='outer', on='DateTime10s')\
    .sort_values('DateTime10s')\
    .ffill()\
    .reset_index(drop=True)
status_df_ts

### Analyzing data before fault

In [ ]:
fault_group_id = 3
fault_group_id_data = grouped_fault_df[grouped_fault_df['FaultGroupID'] == fault_group_id]
fault_time_start = fault_group_id_data['DateTime'].min()
fault_time_end = fault_group_id_data['DateTime'].max()
fault_group_id_data


In [ ]:
scada_status_df = scada_df_gr.merge(status_df_ts, on='DateTime10s', how='outer').sort_values('DateTime10s')


In [ ]:

data_to_analyze = scada_status_df[
    (scada_status_df['DateTime10s'] >= fault_time_start - timedelta(hours=1))
    & (scada_status_df['DateTime10s'] <= fault_time_end + timedelta(hours=1))
].copy()

data_to_analyze['IsFault'] = 0
data_to_analyze.loc[
    (data_to_analyze['DateTime10s'] >= fault_time_start)
    & (data_to_analyze['DateTime10s'] <= fault_time_end),
    'IsFault'
] = 1
data_to_analyze

In [ ]:
# status_df[(status_df['StatusRecordID'] >= 488) & (status_df['StatusRecordID'] <= 514)]

In [ ]:
n_cols = 3
for ifig in range(int(len(data_to_analyze.columns[2:]) / n_cols) + 1):
    fig, axes = plt.subplots(figsize=(22, 5), ncols=n_cols)

    for iax in range(n_cols):
        sns.lineplot(x='DateTime10s', y=data_to_analyze.columns[2 + ifig*n_cols + iax], data=data_to_analyze, hue='IsFault', ax=axes[iax])

In [ ]:
status_df[
    status_df['StatusRecordID'].isin(np.arange(604, 612))
]

# Train model

## Prepare data

In [ ]:
column_mapping = get_sensor_data_info(None)['Column_valid']
column_mapping

In [ ]:
train_start = datetime(2014, 5, 1)
train_end = datetime(2015, 1, 1)

In [ ]:
scada_status_df[(scada_status_df['DateTime'].notna())]

In [ ]:
scada_status_df['IsFault'] = 0
for fault_id, fault_df in grouped_fault_df.groupby('FaultGroupID'):
    fault_time_start = fault_df['DateTime'].min()
    fault_time_end = fault_df['DateTime'].max()
    
    scada_status_df.loc[
        (scada_status_df['DateTime10s'] >= fault_time_start)
        & (scada_status_df['DateTime10s'] <= fault_time_end),
        'IsFault',
    ] = 1
    
    scada_status_df.loc[
        (scada_status_df['DateTime10s'] >= fault_time_start)
        & (scada_status_df['DateTime10s'] <= fault_time_end),
        'FaultGroupID',
    ] = fault_id

scada_status_df

In [ ]:
train_df = scada_status_df[
    (scada_status_df['DateTime'].notna())
    & (scada_status_df['DateTime'] >= train_start)
    & (scada_status_df['DateTime'] <= train_end)
][['DateTime10s', 'HasMissing', *column_mapping.keys(), 'IsFault', 'FaultGroupID']].copy()
train_df = train_df.rename(columns=column_mapping)
train_df

## Train model

In [ ]:
features = [
    *list(column_mapping.values())[3:],
    'HasMissing',
]

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
model = RandomForestClassifier(n_estimators=100, min_samples_split=100)

In [ ]:
model.fit(train_df[features], train_df['IsFault'])

In [ ]:
import pickle

In [ ]:
with open('rf_model.pickle', 'wb') as file:
    pickle.dump(model, file)

In [ ]:
with open('../models/rf_model.pickle', 'rb') as file:
    model_exp = pickle.load(file)

In [ ]:
pred_p = model_exp.predict_proba(train_df[features])[:, 1]

In [ ]:
model_exp.feature_importances_

In [ ]:
model_exp.feature_names_in_